In [ ]:
from math import sqrt
import json
import plotly.express as px
import pandas as pd
from random import randint

In [ ]:
f = open('slippages.json', 'r')

slippages_data = json.loads(f.read())

f.close()

def calculate_mean(l):
  return sum(l)/len(l)

In [ ]:
bid_percentages_means = []
ask_percentages_means = []
time_sample_range = [t for t in range(1, 61)] + [t for t in range(60,300,5)]

for time_sample in time_sample_range:

  bid_percentages = []
  ask_percentages = []

  experiments = 100
  attempts = 100

  for experiment in range(experiments):

    bid_total = 0
    ask_total = 0

    for attempt in range(attempts):
      time_start = randint(1, 1800-time_sample)
      time_end = time_start+time_sample

      # z necessary for 99% confidence is 2.58
      z = 2.58
      mean = 0
      variance = 0
      standard_deviation = 0
      bids_slippages = []
      asks_slippages = []
      data = {}

      bids_slippages = slippages_data['bids'][time_start:time_end]
      asks_slippages = slippages_data['asks'][time_start:time_end]
      
      slippages = bids_slippages

      mean = sum(slippages)/len(slippages)

      for slippage in slippages:
        variance += (slippage - mean)**2
      
      variance /= len(slippages)

      standard_deviation = sqrt(variance)

      sample_size = len(slippages)

      confidence_interval = [mean - z*(standard_deviation)/sqrt(sample_size), mean + z*(standard_deviation)/sqrt(sample_size)]

      data['bids'] = {
        'mean': mean,
        'variance': variance,
        'standard_deviation': standard_deviation,
        'sample_size': sample_size,
        'confidence_interval': confidence_interval
      }

      slippages = asks_slippages

      mean = sum(slippages)/len(slippages)

      for slippage in slippages:
        variance += (slippage - mean)**2
      
      variance /= len(slippages)

      standard_deviation = sqrt(variance)

      sample_size = len(slippages)

      confidence_interval = [mean - z*(standard_deviation)/sqrt(sample_size), mean + z*(standard_deviation)/sqrt(sample_size)]

      data['asks'] = {
        'mean': mean,
        'variance': variance,
        'standard_deviation': standard_deviation,
        'sample_size': sample_size,
        'confidence_interval': confidence_interval
      }

      bid_success = data['bids']['confidence_interval'][0] <= slippages_data['bids'][time_end] <= data['bids']['confidence_interval'][1]
      ask_success = data['asks']['confidence_interval'][0] <= slippages_data['asks'][time_end] <= data['asks']['confidence_interval'][1]

      if bid_success: bid_total += 1
      if ask_success: ask_total += 1

    bid_percentages.append(bid_total/attempts)
    ask_percentages.append(ask_total/attempts)

  bid_percentages_means.append(calculate_mean(bid_percentages))
  ask_percentages_means.append(calculate_mean(ask_percentages))

print(bid_percentages_means, ask_percentages_means)

[0.3075, 0.5777, 0.6061999999999999, 0.5227999999999997, 0.4744, 0.4476999999999998, 0.39750000000000013, 0.38349999999999995, 0.37089999999999995, 0.3699000000000001, 0.38239999999999996, 0.3708000000000002, 0.38730000000000003, 0.3798, 0.3820999999999997, 0.40080000000000005, 0.40659999999999996, 0.39419999999999994, 0.3982, 0.39460000000000006, 0.3804999999999998, 0.38400000000000006, 0.3890000000000001, 0.38830000000000003, 0.3991000000000001, 0.3948, 0.40769999999999984, 0.39520000000000005, 0.39650000000000013, 0.38710000000000006, 0.39699999999999996, 0.38910000000000017, 0.3935, 0.4030000000000003, 0.41340000000000027, 0.4083, 0.4227000000000001, 0.4336, 0.43479999999999985, 0.4280000000000001, 0.42899999999999977, 0.434, 0.4256000000000001, 0.4263999999999999, 0.43279999999999974, 0.42639999999999995, 0.42239999999999983, 0.4217000000000001, 0.4255, 0.42060000000000003, 0.4264, 0.4179999999999999, 0.4134, 0.42269999999999996, 0.4246999999999999, 0.4359, 0.43290000000000006, 0.

In [ ]:
time_start = 1000
time_sample = 60
time_end = time_start+time_sample

bids_slippages = slippages_data['bids'][time_start:time_end]
asks_slippages = slippages_data['asks'][time_start:time_end]

bids_sample_size = len(bids_slippages)
asks_sample_size = len(asks_slippages)

time = [t for t in range(time_start, time_end)]+[t for t in range(time_start, time_end)]
slippages = bids_slippages + asks_slippages
side = ['bid' for bid in bids_slippages] + ['ask' for ask in asks_slippages]

d = { 'time': time, 'slippage': slippages, 'side': side }

df = pd.DataFrame(data=d)

fig = px.line(df, x='time', y='slippage', color='side', title='Slippages over time', labels = 
  {
    'time':'Time (seconds)', 'slippage':'Slippage'
  })
fig.show()

In [ ]:
time_samples = time_sample_range+time_sample_range
percentage_means = bid_percentages_means + ask_percentages_means
side = ['bid' for bid in bid_percentages_means] + ['ask' for ask in ask_percentages_means]

d = { 'time_samples': time_samples, 'percentage_means': percentage_means, 'side': side }

df = pd.DataFrame(data=d)

fig = px.line(df, x='time_samples', y='percentage_means', color='side', title='Time Sample Percentage', labels = 
  {
    'time_samples':'Time Sample (seconds)', 'percentage_means':'Percentage Mean', 'side': 'Side'
  })
fig.show()